# Spectral Decomposition Analysis

This notebook performs spectral decomposition to separate observed spectra into:
- **Event term** (evspec): Source characteristics
- **Station term** (stspec): Site amplification
- **Distance term** (distspec): Geometric spreading (excluding attenuation)

## Workflow
1. Read parameters from `specinp`
2. Organize and filter spectra by distance, time, and SNR
3. Perform iterative inversion to decompose spectra
4. Filter events by minimum station coverage
5. Calibrate magnitudes using low-frequency amplitudes
6. Save results for further analysis

In [ ]:
import os
import sys
import numpy as np
from scipy.io import savemat, loadmat

# Add src to path
sys.path.insert(0, os.path.abspath('../src'))

from specprocess_readpara import specprocess_readpara
from specprocess_organize import specprocess_organize
from specprocess_inversion import specprocess_inversion
from specprocess_plotmag import specprocess_plotmag
from specprocess_specmag import specprocess_specmag

## Read Parameters

Load processing parameters from `specinp` file.

In [ ]:
# NOTE: Use 'specinp_python' to match MATLAB version exactly
# (Change to 'specinp' for different dataset if needed)
P = specprocess_readpara('specinp')
savemat('P.mat', {'P': P})

print("Parameters loaded:")
for key, val in P.items():
    print(f"  {key}: {val}")

## Organize and Filter Spectra

Filter events by geographic bounds, magnitude, distance, SNR, and time constraints.

In [ ]:
name_suffix = 'test'

spec, freq = specprocess_organize(P)
savemat('spec.mat', {'spec': spec, 'freq': freq})
loadmat('spec.mat', {'spec': spec, 'freq': freq})

# Rename with suffix
import shutil
shutil.move('spec.mat', f'spec_{name_suffix}.mat')

print(f"Organized {len(spec)} spectra")
print(f"Frequency range: {freq[0]:.3f} - {freq[-1]:.3f} Hz ({len(freq)} points)")

## Spectral Inversion

Iteratively separate observed spectra into event, station, and distance terms.

In [ ]:
evspec, stspec, distspec = specprocess_inversion(spec, P, freq)
savemat(f'specsolve_{name_suffix}.mat', {
    'evspec': evspec,
    'stspec': stspec,
    'distspec': distspec
})
loadmat(f'specsolve_{name_suffix}.mat', {
    'evspec': evspec,
    'stspec': stspec,
    'distspec': distspec
})

print(f"Inversion complete:")
print(f"  Event spectra: {len(evspec)}")
print(f"  Station spectra: {len(stspec)}")
print(f"  Distance bins: {len(distspec)}")

## Filter Events by Station Coverage

Keep only events recorded by at least `nspecmin` stations (Shearer et al. 2006, Figure 10).

In [ ]:
nspecmin = 4
evspec_save = evspec.copy()

# Filter events with sufficient station coverage
ng = [i for i, ev in enumerate(evspec_save) if ev.get('nspec', 0) >= nspecmin]
evspec = [evspec_save[i] for i in ng]

print(f"Filtered events: {len(evspec_save)} -> {len(evspec)} (nspec >= {nspecmin})")

## Magnitude Calibration

Convert catalog magnitudes to moment magnitude using low-frequency spectral amplitudes (Shearer et al. 2006, Equation 3).

In [ ]:
freqlim = [2, 3]  # Frequency band (Hz) for low-frequency amplitude calculation

# Perform robust regression to get calibration line
y0, slopeline = specprocess_plotmag(evspec, freqlim, nspecmin)

# Magnitude calibration parameters
itype = 2
fmw = 3.0

maginfo = {
    'y0': y0,
    'slope': slopeline,
    'fmw': fmw,
    'itype': itype
}

# Apply calibration to event spectra
evspec = specprocess_specmag(evspec, freqlim, maginfo)
savemat('maginfo.mat', {'maginfo': maginfo})

print(f"Magnitude calibration:")
print(f"  y0 (intercept): {y0:.4f}")
print(f"  slope: {slopeline:.4f}")
print(f"  Frequency band: {freqlim[0]}-{freqlim[1]} Hz")

## Save Results

Event term spectra are now ready for further analysis (e.g., source parameter estimation).

In [ ]:
# Convert list of dicts to MATLAB-compatible struct array
# Define the dtype that matches MATLAB's struct layout
evspec_dtype = np.dtype([
    ('spec', 'O'),
    ('nspec', 'O'),
    ('freq', 'O'),
    ('qid', 'O'),
    ('qtime', 'O'),
    ('qlat', 'O'),
    ('qlon', 'O'),
    ('qdep', 'O'),
    ('qmag', 'O'),
    ('tstar', 'O'),
    ('qmomest', 'O'),
    ('qmagest', 'O'),
    ('qmagresid', 'O')
])

# Create structured array
evspec_array = np.zeros((1, len(evspec)), dtype=evspec_dtype)

for i, ev in enumerate(evspec):
    evspec_array[0, i] = tuple(
        np.asarray(ev.get(field, 0)) for field in [
            'spec', 'nspec', 'freq', 'qid', 'qtime', 'qlat', 'qlon', 
            'qdep', 'qmag', 'tstar', 'qmomest', 'qmagest', 'qmagresid'
        ]
    )

# Same for distspec
if len(distspec) > 0:
    distspec_dtype = np.dtype([
        ('spec', 'O'),
        ('nspec', 'O'),
        ('freq', 'O'),
        ('tt', 'O'),
        ('tstar', 'O')
    ])
    distspec_array = np.zeros((1, len(distspec)), dtype=distspec_dtype)
    for i, ds in enumerate(distspec):
        distspec_array[0, i] = tuple(
            np.asarray(ds.get(field, 0)) for field in ['spec', 'nspec', 'freq', 'tt', 'tstar']
        )
else:
    distspec_array = distspec

# Same for stspec
if len(stspec) > 0:
    stspec_dtype = np.dtype([
        ('spec', 'O'),
        ('nspec', 'O'),
        ('freq', 'O'),
        ('stlat', 'O'),
        ('stlon', 'O'),
        ('stelev', 'O'),
        ('stid', 'O'),
        ('tstar', 'O')
    ])
    stspec_array = np.zeros((1, len(stspec)), dtype=stspec_dtype)
    for i, st in enumerate(stspec):
        stspec_array[0, i] = tuple(
            np.asarray(st.get(field, 0)) for field in ['spec', 'nspec', 'freq', 'stlat', 'stlon', 'stelev', 'stid', 'tstar']
        )
else:
    stspec_array = stspec

savemat(f'evspec_{name_suffix}.mat', {
    'freq': freq,
    'evspec': evspec_array,
    'y0': y0,
    'slopeline': slopeline,
    'distspec': distspec_array,
    'stspec': stspec_array,
    'freqlim': freqlim
})

print(f"\nResults saved to evspec_{name_suffix}.mat")
print(f"Event spectra ready for source parameter analysis!")
print(f"\nSummary:")
print(f"  - {len(evspec)} events with nspec >= {nspecmin}")
print(f"  - {len(stspec)} station terms")
print(f"  - {len(freq)} frequency points ({freq[0]:.3f} - {freq[-1]:.3f} Hz)")
print(f"  - Magnitude calibration: y0={y0:.4f}, slope={slopeline:.4f}")